# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [57]:
from resolve import *

In [105]:
from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/summer/version3/multi_modal_davis_rand")

if "metadata.data.provenance_cols" in df.columns:
    df["metadata.data.provenance_cols"] = df["metadata.data.provenance_cols"].apply(lambda x: x[0])
if "metadata.data.drug_features" in df.columns:
    df = df.drop(columns=["metadata.data.drug_features", "metadata.data.target_features"])

df

2025-07-27 18:37:11,651 - mb_vae_dti.validating.analysis - INFO - Successfully loaded 661 results with 40 columns


,experiment_name,best_val_loss,best_epoch,trainable_params,timestamp,metadata.training.max_epochs,metadata.training.early_stopping_patience,metadata.training.scheduler,metadata.training.learning_rate,metadata.data.load_in_memory,...,metrics.val/ci,metrics.val/mse,metrics.val/pearson,metrics.val/r2,metrics.val/rmse,timing.total_training_time,timing.avg_time_per_epoch,timing.total_epochs,timing.min_epoch_time,timing.max_epoch_time
0,multimodal_DAVIS_rand_b02c0032_1753130126,0.246789,69,15310978,1.753130e+09,100,10,one_cycle,0.0010,True,...,0.871983,0.258119,0.813023,0.646458,0.671788,325.406115,4.067576,80,3.749988,4.351934
1,multimodal_DAVIS_rand_b04c0048_1753157319,0.220964,43,2901888,1.753158e+09,100,10,one_cycle,0.0010,True,...,0.869447,0.239715,0.822206,0.671666,0.625248,457.531310,8.472802,54,7.989107,9.527689
2,multimodal_DAVIS_rand_b01c0024_1753120441,0.238873,90,15048322,1.753122e+09,100,10,one_cycle,0.0005,True,...,0.883737,0.241255,0.819131,0.669553,0.620584,1474.125477,14.741255,100,13.943223,15.697541
3,multimodal_DAVIS_rand_b06c0027_1753133862,0.210346,67,36836352,1.753134e+09,100,10,one_cycle,0.0010,True,...,0.877291,0.214484,0.842180,0.706225,0.616258,357.194001,4.579410,78,4.396237,4.669112
4,multimodal_DAVIS_rand_b01c0061_1753153725,0.270694,94,39483264,1.753156e+09,100,10,one_cycle,0.0020,True,...,0.873386,0.272349,0.793306,0.626964,0.633022,2673.646842,26.736468,100,25.610475,28.567416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,multimodal_DAVIS_rand_b04c0079_1753185646,0.262114,85,7341698,1.753186e+09,100,10,one_cycle,0.0005,True,...,0.868939,0.265659,0.802612,0.636131,0.668769,510.334897,5.315989,96,4.962592,5.576232
657,multimodal_DAVIS_rand_b01c0039_1753133330,0.241932,95,26215682,1.753134e+09,100,10,one_cycle,0.0005,True,...,0.879260,0.242270,0.818808,0.668167,0.621534,1047.371928,10.473719,100,9.911473,11.015590
658,multimodal_DAVIS_rand_b00c0024_1753111488,0.191585,77,7596544,1.753112e+09,100,10,one_cycle,0.0020,True,...,0.894047,0.196720,0.858306,0.730556,0.631963,336.738482,3.826574,88,3.579693,4.149179
659,multimodal_DAVIS_rand_b01c0072_1753168772,0.205158,86,2044930,1.753169e+09,100,10,one_cycle,0.0005,True,...,0.887221,0.205752,0.847790,0.718185,0.563668,386.079619,3.980202,97,3.663367,4.241402


In [106]:
varying_attrs = get_varying_attributes(df)
varying_attrs.remove('metadata.logging.experiment_name')
varying_attrs.extend(["best_val_loss", 'best_epoch', 'trainable_params', "timing.total_training_time"])
varying_attrs

2025-07-27 18:37:11,809 - mb_vae_dti.validating.analysis - INFO - Found 8 varying attributes: ['metadata.training.learning_rate', 'metadata.data.batch_size', 'metadata.logging.experiment_name', 'metadata.model.encoder_kwargs.hidden_dim', 'metadata.model.encoder_kwargs.n_layers', 'metadata.model.embedding_dim', 'metadata.model.encoder_type', 'metadata.model.aggregator_type']


['metadata.training.learning_rate',
 'metadata.data.batch_size',
 'metadata.model.encoder_kwargs.hidden_dim',
 'metadata.model.encoder_kwargs.n_layers',
 'metadata.model.embedding_dim',
 'metadata.model.encoder_type',
 'metadata.model.aggregator_type',
 'best_val_loss',
 'best_epoch',
 'trainable_params',
 'timing.total_training_time']

In [107]:
subdf = get_top_performers(df, n_top=5)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(df["timing.total_training_time"].mean())
print(subdf["trainable_params"].mean() / 1000000)
subdf

888.384538050679
7.682996


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,aggregator_type,best_val_loss,best_epoch,trainable_params,total_training_time
49,0.0020,16,128,1,768,resnet,concat,0.175165,92,1746560,1018.947210
527,0.0010,32,512,2,512,resnet,attentive,0.179157,78,24888322,642.109032
186,0.0020,16,256,1,1024,resnet,concat,0.180105,75,5294336,856.976062
403,0.0005,16,256,1,512,resnet,attentive,0.184622,73,4243714,892.207972
251,0.0020,64,128,2,512,resnet,concat,0.185169,71,2242048,306.365087


In [116]:
a = 0.83483
b = 0.82954
c = 0.83459
d = 0.83648
e = 0.83343


total = a + b + c + d + e
print(total / 5)

0.833774
